# 💡 How2DDDA: **"Implicit Function Theory — from Mathematics to Automated Machine Scientist Applications"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**隐函数理论 - 机器科学家应用**

In this notebook, we will walk through the theoretical and computational foundation of **dimensional analysis**, with a focus on the **Buckingham Pi theorem**. You will learn:

1. **The motivation behind dimensional analysis**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **How to construct the dimensional matrix (D-matrix)**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **How to compute Π-groups using null space techniques**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **How to interpret and validate Π-groups**  
   Learn to assess whether derived groups make physical and computational sense.

5. **How to extend the method toward data-driven workflows**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: DDDA Project, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full DDDA pipeline.*


## 1 · Implicit Function Theorem — General Multivariable Form & Geometry

- 一段“严格表述”的占位（multi-variate, multi-dependent 统一形式）。

- 一段“直观几何解读”的占位（水平集、法向/切向、图像表示）。

- 统一符号表与可复用的记号约定（后续章节复用）。

- 2 个代码骨架：Jacobian 可逆性检查（SymPy）+ 局部线性近似的可视化占位（稍后补画图）。

### 域的直积分解与符号

假如我们从**学科数据库**或**复杂的实验机器人**中随机获取了一组物理变量，我们希望回答两个核心问题：

* 这些变量之间**是否存在物理关系**？
* 如果存在，**这种关系的数学形式**是什么？

此时，我们可以利用**隐函数定理**的性质，实现这一过程的自动化判定与分析。

---

### 1). 获得变量集合(物理量层面)

假设通过上述途径，我们获得了 $n$ 个独立的物理量，记为：

$$Q_1, Q_2, \dots, Q_n$$

其中每个 $Q_i$ 都是定义在某个物理域的实数变量（如速度、温度、压力等）。

我们可以将变量总数写作：

$$
n = m + k
$$

其中：

* $m$：自变量（independent variables）的维数；
* $k$：因变量（dependent variables）的维数。

> 在这一阶段，我们**不预先指定**哪些 $Q_i$ 属于自变量或因变量，后续将引入自动判定方法。

---

### 2) 打包成向量（数学化）

为便于统一处理，将所有物理量按顺序排列成一个 $n$ 维列向量：

$$
z := (Q_1, Q_2, \dots, Q_{m+k})^\mathrm{T} \in \mathbb{R}^{m+k}.
$$

这里的 $z$ 是“所有输入变量”的整体表示。

---

### 3) 变量分块（隐函数定理的入口）

在数学建模中，我们可以将 $z$ 划分为两个互不相交的子块：

$$
x := (Q_{i_1}, \dots, Q_{i_m}) \in \mathbb{R}^m, 
\quad 
y := (Q_{j_1}, \dots, Q_{j_k}) \in \mathbb{R}^k,
$$

其中：

$$
\{i_1, \dots, i_m\} \cup \{j_1, \dots, j_k\} = \{1, \dots, m+k\}, \quad \{i_\alpha\} \cap \{j_\beta\} = \varnothing.
$$

* $x$：**自变量块**（independent variables）
* $y$：**因变量块**（dependent variables）

> 在我们的自动方法中，这一分块将由算法根据可解性条件自动判定，而非人工指定。

---

### 4) 建立隐函数形式的约束方程

系统的物理规律可表示为 $k$ 条独立方程：

$$
F(x, y) = 0, \quad F : \mathbb{R}^m \times \mathbb{R}^k \to \mathbb{R}^k.
$$

这里：

* **定义域**：自变量块 $x$ 与因变量块 $y$ 共同构成 $\mathbb{R}^m \times \mathbb{R}^k$；
* **值域**：$\mathbb{R}^k$，因为有 $k$ 条方程，每条方程的输出是一个实数，合起来就是 $k$ 维向量。

在**自动判定**的语境下，我们也可以直接写作：

$$
F(Q_1, Q_2, \dots, Q_{m+k}) = 0
$$

表示这些物理量之间存在某种隐式关系。

---
